Anmerkung: 
habe den Code gerade zusammenkopiert (27.05.) und mache jetzt Pause, sortiere dann morgen alles um das mit der linearen regression nochmal auszuprobieren!

In [ ]:
# import packages
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import csv

In [ ]:
# load data: is "5_has_wettercode" the latest version?

wetter = pd.read_csv("/workspaces/bakery_sales_prediction/sourcedata/cleaned_data/5_has_wettercode.csv")
print(wetter.head(), "\n")

## 1. Wettercodes:

### Wetterdcodes decodieren und in neue Klassen einteilen

In [ ]:
# Print the frequency of each value of the weather codes (which ones occure?)
value_counts = wetter['Wettercode'].value_counts().sort_index()
print(value_counts)

In [ ]:
# Dictionary with weather codes and their descriptions from http://www.seewetter-kiel.de/seewetter/daten_symbole.htm
weather_code_descriptions = {
    0: "Bewoelkungsentwicklung nicht beobachtet",
    3: "Zunehmende Bewoelkung",
    5: "Trockener Dunst",
    10: "Feuchter Dunst",
    17: "Gewitter mit hoerbarem Donner, aber kein Niederschlag an der Station",
    20: "Spruehregen oder Schneegriesel hat aufgehoert",
    21: "Regen hat aufgehoert",
    22: "Schneefall hat aufgehoert",
    28: "Nebel hat sich aufgeloest",
    45: "Nebel, Himmel ist nicht erkennbar, Nebel unveraendert",
    49: "Nebel mit Reifbildung, Himmel ist nicht erkennbar",
    53: "Durchgehender maeßiger und nicht gefrierender Spruehregen",
    61: "Durchgehender leichter und nicht gefrierender Regen",
    63: "Durchgehender maeßiger nicht gefrierender Regen",
    65: "Durchgehender starker und nicht gefrierender Regen",
    68: "Leichter Schneeregen",
    69: "Maeßiger oder starker Schneeregen",
    71: "Durchgehender leichter Schneefall",
    73: "Durchgehender maeßiger Schneefall",
    75: "Durchgehender starker Schneefall",
    77: "Schneegrieseln mit oder ohne Nebel",
    79: "Eiskoerner (gefrorene Regentropfen)",
    95: "Leichtes oder maeßiges Gewitter mit Regen oder Schnee"
}

# Function to get the description of a weather code
def get_weather_description(code):
    return weather_code_descriptions.get(code, "Description not available")

get_weather_description(53)

In [ ]:
# new classes for weathercodes in a dictionary:
new_weather_code_descriptions = {
    0 : "nicht_beobachtet",
    1 : "Wolken",
    2 : "Dunst_oder_Nebel",
    3 : "sich_ankuendigender_Regen",
    4 : "Gewitter",
    5 : "Boeen_und_Sturm",
    6 : "Niederschlag_nass_leicht", # Regen # leicht # inkl. 'Niederschlag hat aufgehört', weil es ja nicht den ganzen Tag lang 'aufgehört' haben kann, sondern dann vermutlich vorher geregnet hat
    7 : "Niederschlag_nass_stark", # mäßiger oder starker Regen
    8 : "Niederschlag_trocken", # Schnee; leichtes oder mäßiges Schneetreiben
    9 : "Schneesturm_Hagel", # starkes Schneefegen
    10 : "Sandsturm"
}

In [ ]:
# Define lists of codes for each weather class
nicht_beobachtet =  [0] # 0
Wolken = [1, 2, 3] # 1
Dunst_oder_Nebel = [4, 5, 6, 7, 8, 9, 10, 11, 12, 28, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49] # 2
sich_ankuendigender_Regen = [14, 15, 16] # 3
Gewitter = [13, 17, 29, 91, 92, 93, 94, 95, 96, 97, 98, 99] # 4
Boeen_und_Sturm = [18, 19] # 5
Niederschlag_nass_leicht = [20, 21, 23, 24, 25, 27, 50, 51, 56, 58, 60, 61, 66, 68, 80, 83, 87] # 6
Niederschlag_nass_stark = [52, 53, 54, 55, 57, 59, 62, 63, 64, 65, 67, 69, 81, 82, 84, 88] # 7
Niederschlag_trocken = [22, 26, 36, 38, 70, 72, 73, 74, 75, 77, 78, 85, 86] # 8
Schneesturm_Hagel = [37, 39, 76, 79, 89, 90] # 9
Sandsturm = [30, 31, 32, 33, 34, 35] # 10


In [ ]:
# Create a combined dictionary for mapping original codes to new numeric codes
weather_code_mapping = {}

# Combine the lists and dictionary
weather_classes = {
    "nicht_beobachtet" : nicht_beobachtet,
    "Wolken" : Wolken,
    "Dunst_oder_Nebel" : Dunst_oder_Nebel,
    "sich_ankuendigender_Regen" : sich_ankuendigender_Regen,
    "Gewitter" : Gewitter,
    "Boeen_und_Sturm" : Boeen_und_Sturm,
    "Niederschlag_nass_leicht" : Niederschlag_nass_leicht,
    "Niederschlag_nass_stark" : Niederschlag_nass_stark,
    "Niederschlag_trocken" : Niederschlag_trocken,
    "Schneesturm_Hagel" : Schneesturm_Hagel,
    "Sandsturm" : Sandsturm
}

# Iterate over the new_weather_code_descriptions to build the mapping dictionary
for new_code, description in new_weather_code_descriptions.items():
    if description in weather_classes:
        for code in weather_classes[description]:
            weather_code_mapping[code] = new_code

# Map the weather codes to the new numeric codes
wetter['Wetterklasse'] = wetter['Wettercode'].map(weather_code_mapping)

# Replace NaN values with -9999 for codes not in the mapping
wetter['Wetterklasse'] = wetter['Wetterklasse'].fillna(-9999)

# Print the updated DataFrame
print(wetter)

### Klassen mit zu wenig Objekten herausfiltern

In [ ]:
# Find a threshold which classes to include! (small object numbers per class might be too little to include into the model --> find threshold)

# Calculate value counts
value_counts = wetter['Wettercode'].value_counts().sort_index()

# Calculate total number of occurrences
total_counts = value_counts.sum()

# Define the threshold percentage (e.g., 5%)
threshold_percentage = 0.05

# Calculate the minimum number of occurrences to be included
threshold_count = total_counts * threshold_percentage

# Filter weather codes based on the threshold count
filtered_codes = value_counts[value_counts >= threshold_count]

print("Total occurrences per weather code:")
print(value_counts)
print(f"\nFiltered weather codes (above threshold {threshold_percentage}):")
print(filtered_codes)

In [ ]:
# Datensatz anpassen und rows löschen? 

## 2. Niederschlagsdaten einfügen

In [ ]:
# read in weather data
weather = pd.read_csv("/workspaces/bakery_sales_prediction/sourcedata/wetter.csv")
weather["Datum"] = pd.to_datetime(weather["Datum"])

# Define the path to additional weather data from DWD
file_path = "/workspaces/bakery_sales_prediction/sourcedata/produkt_klima_tag_19740101_20231231_02564.txt"

# Read the file into a DataFrame, parse dates, and treat -999 as NaN
prec = pd.read_csv(file_path, sep=';', na_values='-999', parse_dates=['MESS_DATUM'])

# Rename the MESS_DATUM column to Datum
prec.rename(columns={'MESS_DATUM': 'Datum'}, inplace=True)
prec.rename(columns={" RSK": "Niederschlag"}, inplace = True)
# prec.rename(columns={"  FM": "Windgeschwindigkeit_m-per-s"}, inplace = True)

# Select relevant columns
prec = prec[["Datum", "Niederschlag"]]

# Display the first few rows of the DataFrame
print(prec.head())

# Merge both datasets on the Datum column
weather_data = pd.merge(weather, prec, on='Datum', how='inner')

# Display the merged DataFrame
print(weather_data)

## 3. Wochentage

In [ ]:
# extract date features
weather_data['year'] = weather_data['Datum'].dt.year
weather_data['month'] = weather_data['Datum'].dt.month
weather_data['week'] = weather_data['Datum'].dt.isocalendar().week
weather_data['day_of_week'] = weather_data['Datum'].dt.dayofweek
weather_data['day_of_month'] = weather_data['Datum'].dt.day
print(weather_data.head())


In [ ]:
# define the season
def get_season(month):
	    if month in [12, 1, 2]:
	        return 'winter'
	    elif month in [3, 4, 5]:
	        return 'spring'
	    elif month in [6, 7, 8]:
	        return 'summer'
	    else:
	        return 'autumn'
	
weather_data['season'] = weather_data['month'].apply(get_season)

# Create binary features for seasons
# probably to generic for predictions?
weather_data['winter'] = (weather_data['season'] == 'winter').astype(int)
weather_data['spring'] = (weather_data['season'] == 'spring').astype(int)
weather_data['summer'] = (weather_data['season'] == 'summer').astype(int)
weather_data['autumn'] = (weather_data['season'] == 'autumn').astype(int)

print(weather_data.head())


## 4. Saisonale Abweichungen:

In [ ]:
# check what exactly this means and does! (Suggestion by ChatGPT)
# Create rolling averages for temperature, precipitation and and wind speed
# weather_data['temp_7d_avg'] = weather_data['Temperatur'].rolling(window=7).mean()
# weather_data['wind_7d_avg'] = weather_data['Windgeschwindigkeit'].rolling(window=7).mean()
# weather_data['prec_7d_avg'] = weather_data['Niederschlag'].rolling(window=7).mean()

# Create lag features
# weather_data['temp_lag_1'] = weather_data['Temperatur'].shift(1)
# weather_data['wind_lag_1'] = weather_data['Windgeschwindigkeit'].shift(1)
# weather_data['prec_lag_1'] = weather_data['Niederschlag'].shift(1)

# Fill NA values created by rolling and lag features
# weather_data.fillna(method='bfill', inplace=True)

# Normalize the weather features
# from sklearn.preprocessing import StandardScaler

# weather_features = ['Temperatur', 'Windgeschwindigkeit', 'Niederschlag', 'temp_7d_avg', 'wind_7d_avg', 'temp_lag_1', 'wind_lag_1', 'prec_7d_avg', 'prec_lag_1']
# scaler = StandardScaler()
# weather_data[weather_features] = scaler.fit_transform(weather_data[weather_features])
# print(weather_data.head())


In [ ]:
# Open ToDos:
# Create categories for temperature depending on season
# or calculate the deviation from monthly temperature
# or define warm / cold day depending on the season (e.g. in winter 10°C is a warm day, in summer that would be a cold day)

In [ ]:
# Define thresholds for extreme weather conditions
VERY_HOT_THRESHOLD = 25  # Example: temperature > 25 degrees Celsius
VERY_COLD_THRESHOLD = 0  # Example: temperature < 0 degrees Celsius
VERY_RAINY_THRESHOLD = 10 # Example: precipitation > 10 mm
HIGH_WIND_THRESHOLD = 15  # Example: wind speed > 15 m/s
SUNNY_DAY_THRESHOLD = 2  # Example: cloudiness <= 2 - how is the parameter defined?

# Create binary features for extreme weather conditions
weather_data['very_hot'] = (weather_data['Temperatur'] > VERY_HOT_THRESHOLD).astype(int)
weather_data['very_cold'] = (weather_data['Temperatur'] < VERY_COLD_THRESHOLD).astype(int)
weather_data['very_rainy'] =  (weather_data['Niederschlag'] > VERY_RAINY_THRESHOLD).astype(int)
weather_data['high_wind'] = (weather_data['Windgeschwindigkeit'] > HIGH_WIND_THRESHOLD).astype(int)
weather_data['sunny_days'] = (weather_data['Bewoelkung'] <= SUNNY_DAY_THRESHOLD).astype(int)

# Display the first few rows to check all features
print(weather_data)


In [ ]:
# Calculate "feels like" temperature
weather_data["Windgeschwindigkeit_km_h"] = weather_data["Windgeschwindigkeit"] * 3.6
weather_data["feels_like_temperature"] = 13.12 + 0.6215 * weather_data["Temperatur"] - 11.37 * weather_data["Windgeschwindigkeit_km_h"]**0.16 + 0.3965*weather_data["Temperatur"]*weather_data["Windgeschwindigkeit_km_h"]**0.16
print(weather_data.head())

# export file again
# weather_data.to_csv("Files/weather_data.csv", index = False)


## Ergebnis speichern:

In [ ]:
 # Save the updated data to new CSV files
    data.to_csv('../sourcedata/cleaned_data/6_weather_parameters.csv', sep=',', index=False)